In [1]:
import tensorflow as tf
from tensorflow.python import debug as tf_debug

In [2]:
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  1.5.0


In [3]:
import os
from os.path import join
import json
import random
import itertools
import re
import datetime
import numpy as np
from scipy import ndimage
import pylab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
import cv2
import glob

In [13]:
import string 
from collections import Counter
characters = sorted(list(set(Counter(string.ascii_letters).keys())))
digits = sorted(list(set(Counter(string.digits).keys())))
whitespace = sorted(list(set(Counter(string.whitespace).keys())))
characters.append(digits)
characters.append(whitespace)
print(characters)


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '\t', '\n', '\x0b', '\x0c', '\r', ' ']


# Input Data Generator

In [6]:
from os import listdir
from os.path import isfile

class Dataset:
    def __init__(self, directory_path, img_height, img_width, batch_size):
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        
        # get sample filenames
        img_directory_path = directory_path + '/img/'
        label_directory_path = directory_path + '/json/' 
        self.img_filenames = []
        self.label_filenames = []
        self.imgs = []
        for filename in os.listdir(img_directory_path):
            img_filenames.append(img_directory_path + filename)
            imgs.append(cv2.imread(img_directory_path + filename, 0))
        for filename in os.listdir(label_directory_path):
            label_filenames.append(label_directory_path + filename)
         
        # store filenames as tensors
        self.img_filename_tensors = tf.constant(img_filenames)
        self.label_tensors = tf.constant(label_filenames)
        self.img_tensors = tf.constant(np.asarray(imgs))
        self.dataset = tf.data.Dataset.from_tensor_slices((self.img_tensors, self.label_tensors))
    
    def next_batch(self, batch_size):
        return
 
    
# get sample filenames
directory_path = './dataset/dummy'
img_directory_path = directory_path + '/img/'
label_directory_path = directory_path + '/json/' 
img_filenames = []
label_filenames = []
imgs = []
for filename in os.listdir(img_directory_path):
    img_filenames.append(img_directory_path + filename)
    imgs.append(cv2.imread(img_directory_path + filename, 0))
for filename in os.listdir(label_directory_path):
    label_filenames.append(label_directory_path + filename)

# store filenames as tensors
imgs = tf.constant(np.asarray(imgs))
img_tensors = tf.constant(img_filenames)
label_tensors = tf.constant(label_filenames)

#dataset = Dataset('./dataset/dummy', 256, 256, 1)   
#tf_dataset = tf.data.Dataset.from_tensor_slices((dataset.img_tensors, dataset.label_tensors))   
tf_dataset = tf.data.Dataset.from_tensor_slices((imgs, label_tensors))   
iterator = tf.data.Iterator.from_structure(tf_dataset.output_types, tf_dataset.output_shapes)
next_sample = iterator.get_next()
init = (tf.global_variables_initializer(), iterator.make_initializer(tf_dataset))

sess = tf.InteractiveSession()
#sess = tf_debug.LocalCLIDebugWrapperSession(sess)
sess.run(init)
while True:
    try: 
        sample = sess.run(next_sample)
        print(sample)
    except tf.errors.OutOfRangeError:
        print("End of training dataset.")
        break
        

(array([[14, 13, 13, ..., 13, 13, 13],
       [13, 13, 13, ..., 13, 13, 13],
       [13, 14, 14, ..., 13, 12, 12],
       ...,
       [ 6,  6,  6, ..., 11, 11, 11],
       [ 6,  6,  6, ..., 11, 11, 11],
       [ 6,  6,  6, ..., 11, 11, 11]], dtype=uint8), b'./dataset/dummy/json/andrewlloydwebberandtimrice_jesuschristsuperstar_0001.json')
(array([[ 6,  6,  6, ...,  7,  6,  6],
       [ 6,  6,  6, ...,  7,  6,  6],
       [ 6,  6,  6, ...,  7,  7,  7],
       ...,
       [80, 78, 76, ...,  4,  4,  4],
       [82, 77, 73, ...,  4,  4,  4],
       [85, 77, 70, ...,  4,  4,  4]], dtype=uint8), b'./dataset/dummy/json/chic_cestchic_0001.json')
(array([[3, 2, 2, ..., 3, 3, 2],
       [3, 2, 2, ..., 3, 3, 2],
       [3, 3, 3, ..., 3, 3, 2],
       ...,
       [3, 3, 2, ..., 2, 2, 2],
       [3, 3, 2, ..., 2, 2, 2],
       [4, 3, 1, ..., 2, 2, 2]], dtype=uint8), b'./dataset/dummy/json/ciare_likeaboy_0002.json')
(array([[11, 11, 11, ..., 13, 14, 14],
       [11, 11, 11, ..., 13, 14, 14],
       [

# Network Layers & Helper Functions

In [38]:
# Init Weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=.1)
    return tf.Variable(init_random_dist)

# Init Bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape);
    return tf.Variable(init_bias_vals)

# Conv2D
# x: [batch size, height, width, # of channels]
# W: [height filter, width filter, # of input channels, # of output channels]
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# Pooling
# x: [batch size, height, width, # of channels]
# ksize: size of the window for each dimension of the input tensor
# strides: 
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1] , strides=[1,2,2,1], padding='SAME')

# Convolutional Layer
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

# Normal (Fully Connected) Layer
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return (tf.matmul(input_layer, W)+b)

# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 65536])
y_true = tf.placeholder(tf.float32, shape=[None, len(letters)])

# Layers
x_image = tf.reshape(x, [-1,256,256,1])
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32])
pool_1 = max_pool_2by2(convo_1)
convo_2 = convolutional_layer(pool_1, shape=[5,5,32,64])
pool_2 = max_pool_2by2(convo_2)
flat_1 = tf.reshape(pool_2, [-1,256*256*64])
full_layer_one = tf.nn.relu(normal_full_layer(flat_1, 1024))

# Dropout
# used to prevent overfitting
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)
y_pred = normal_full_layer(full_one_dropout, len(letters))

# loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()


# Old Work

In [36]:
from os import listdir
from os.path import isfile

def images_to_tensors(img_paths):
    img_queue = tf.train.string_input_producer(img_paths)  
    reader = tf.WholeFileReader()
    keys, values = reader.read_up_to(img_queue, tf.cast(img_queue.size(), dtype=tf.int64))
    images = tf.Variable([])
    fn = lambda x: images.append(tf.image.decode_png(x))
    tf.map_fn = (fn, values)
    fn2 = lambda x: print(x)
    tf.map_fn(fn2, images)
    return images

def labels_to_tensors(directory_path):
    label_strings = []
    for filename in os.listdir(directory_path):
        artist = json.load(open(directory_path + filename, 'r'))['artist']
        album = json.load(open(directory_path + filename, 'r'))['album']
        label = artist + '\n' + album;
        print(label)
        label_strings.append(label)
    label_tensors = tf.Variable(label_strings)
    return label_tensors

class Dataset:
    def __init__(self, directory_path, img_height, img_width, batch_size):
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        
        img_directory_path = directory_path + '/img/'
        label_directory_path = directory_path + '/json/' 
        print('Getting image data from path ', img_directory_path)

        img_paths = [f for f in listdir(img_directory_path) if isfile(join(img_directory_path, f))]
        print(img_paths)
        self.images= images_to_tensors(img_paths)
        print(self.images)
        print('Getting label data from path ', label_directory_path)
        self.labels = labels_to_tensors(label_directory_path)
        
    def next_batch(self, batch_size):
        perm0 = numpy.arange(batch_size)
        numpy.random.shuffle(perm0)
        self._images = self.images[perm0]
        self._labels = self.labels[perm0]
        return self._images[start:end], self._labels[start:end]
    
    def print(self):
        print('Printing Image Dataset')
        fn = lambda x: print(x)
        tf.map_fn(fn,self.images)

In [37]:
dataset_train = Dataset('./dataset/train', 256, 256, 25)

Getting image data from path  ./dataset/train/img/
['andrewlloydwebberandtimrice_jesuschristsuperstar_0002.JPG', 'bg_choppercityintheghetto_0001.JPG', 'bg_choppercityintheghetto_0002.JPG', 'bjork_debut_0001.JPG', 'bjork_debut_0002.JPG', 'blackflag_damaged_0001.JPG', 'blackflag_damaged_0002.JPG', 'blackflag_mywar_0001.JPG', 'blackflag_mywar_0002.JPG', 'catstevens_foreigner_0001.JPG', 'catstevens_foreigner_0002.JPG', 'catstevens_greatesthits_0001.JPG', 'catstevens_greatesthits_0002.JPG', 'charlieparker_bird-thesavoyrecordings_0001.JPG', 'charlieparker_bird-thesavoyrecordings_0002.JPG', 'charlieparker_bird-thesavoyrecordings_0003.JPG', 'charlieparker_bird-thesavoyrecordings_0004.JPG', 'cher_theverybestofcher_0001.JPG', 'cher_theverybestofcher_0002.JPG', 'chic_cestchic_0002.JPG', 'chic_chic_0001.JPG', 'chic_chic_0002.JPG', 'ciare_likeaboy_0001.JPG', 'clipping_splendor&misery_0001.JPG', 'clipping_splendor&misery_0002.JPG', 'commodores_midnightmagic_0001.JPG', 'commodores_midnightmagic_0002.

TypeError: 'tuple' object is not callable

## Helper Functions

In [ ]:
# Init Weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=.1)
    return tf.Variable(init_random_dist)

# Init Bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape);
    return tf.Variable(init_bias_vals)

# Conv2D
# x: [batch size, height, width, # of channels]
# W: [height filter, width filter, # of input channels, # of output channels]
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# Pooling
# x: [batch size, height, width, # of channels]
# ksize: size of the window for each dimension of the input tensor
# strides: 
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1] , strides=[1,2,2,1], padding='SAME')

# Creating Network Layers

In [ ]:
# Convolutional Layer
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

# Normal (Fully Connected) Layer
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return (tf.matmul(input_layer, W)+b)

# Define Network Architecture

In [ ]:
# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 65536])
y_true = tf.placeholder(tf.float32, shape=[None, len(letters)])

# Layers
x_image = tf.reshape(x, [-1,28,28,1])
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32])
pool_1 = max_pool_2by2(convo_1)
convo_2 = convolutional_layer(pool_1, shape=[5,5,32,64])
pool_2 = max_pool_2by2(convo_2)
flat_1 = tf.reshape(pool_2, [-1,256*256*64])
full_layer_one = tf.nn.relu(normal_full_layer(flat_1, 1024))

# Dropout
# used to prevent overfitting
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)
y_pred = normal_full_layer(full_one_dropout, len(letters))

# Loss Function

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))

# Optimizer

In [ ]:
sess = tf.Session()

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
data = Dataset('dataset/train/', 256, 256, 50)
print(data)

In [ ]:
steps = 5000
data = Dataset('dataset/train/', 256, 256, 50)
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = Dataset.next_batch(25)
        
        sess.run(train, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5})
        
        if i % 100 == 0:
            print("On Step: ".format(i))
            print("accuracy");
            
            matches = tf.equal(tf.argmax(y-pred,1)), tf.argmax(y_true, 1)
            acc tf.reduce_mean(tf.cast(matches, tf.float32))
            print (sess.run(acc,feed-dict={x:Dataset.images, y:Dataset.labels, hold_prob:1.0}))
            print('\n')